# 🚀 databricks-scaffold: Complete Showcase

Productivity booster functions for using Polars on Databricks. Ease the pain of working with small data on Databricks. 

This notebook demonstrates the complete suite of features:
1. **PySpark Utilities**: Missing quality-of-life functions for PySpark DataFrames (`clean_column_names`, `keep_duplicates`, `frame_shape`, `apply_column_comments`).
2. **The "Polars Sandwich"**: Fast, zero-memory-spike conversions between distributed PySpark DataFrames and local Polars DataFrames using Unity Catalog Volumes.
3. **Checkpointing**: Easy saving and loading to persistent Unity Catalog 🛜volume or ⚡ephemeral local driver storage.
4. **Timestamp Fixes**: Silent, automatic handling of nanosecond to millisecond conversions to prevent Spark crashes.

In [0]:
import polars as pl
from pyspark.sql import SparkSession, functions as F, types as T
from datetime import datetime, timedelta
import uuid

import sys
import os
from pathlib import Path
repo_root = Path(os.getcwd()).parent
src_path = str(repo_root / "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Import databricks-scaffold core and utilities
from databricks_scaffold import (
    VolumeSpiller, 
    frame_shape, 
    clean_column_names, 
    keep_duplicates, 
    apply_column_comments
)

# --- Configuration ---
IS_DEV = True # In Dev, we preserve volumes; in Prod, we wipe them
CATALOG = "main"
SCHEMA = "default"
VOL_NAME = f"showcase_vol_{uuid.uuid4().hex[:6]}"

# Initialize Spark and VolumeSpiller
spark = SparkSession.builder.appName("ScaffoldShowcase").getOrCreate()
spill = VolumeSpiller(spark, CATALOG, SCHEMA, VOL_NAME, is_dev=IS_DEV)

print(f"✅ Initialization complete. Using volume: {spill.full_name}")

## 🛠️ Part 1: PySpark Utility Functions
First, let's create some messy PySpark data and clean it up using the scaffold's utility functions.

In [0]:
# Generate data with messy column names and intentional duplicates
data = [
    (1, "Alice", "alice@test.com", datetime(2024, 1, 1)),
    (2, "Bob",   "bob@test.com",   datetime(2024, 1, 2)),
    (1, "Alice", "alice@test.com", datetime(2024, 1, 1)), # Duplicate!
    (3, "Carol", "carol@test.com", datetime(2024, 1, 3))
]

# Note the spaces and special characters in column names
df_messy = spark.createDataFrame(data, ["User ID #", "First Name!", "Email Address", "Created At"])

print("--- Original Messy DataFrame ---")
df_messy.show()

# 1. clean_column_names()
df_clean = clean_column_names(df_messy)
print("--- After clean_column_names() ---")
print(df_clean.columns)

# 2. keep_duplicates()
df_dupes = keep_duplicates(df_clean, subset=["User_ID", "First_Name"])
print("--- Duplicated Rows Found ---")
df_dupes.show()

# 3. frame_shape()
print("--- DataFrame Shape ---")
shape = frame_shape(df_clean)

In [0]:
# 4. apply_column_comments()
# Let's save the table to Unity Catalog to apply comments to it
table_name = f"{CATALOG}.{SCHEMA}.scaffold_test_table"

(df_clean.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(table_name))

comments = {
    "User_ID": "Unique identifier for the user.",
    "First_Name": "User's given name.",
    "Email_Address": "Contact email.",
    "Created_At": "Account creation timestamp."
}

print("--- Applying Column Comments ---")
apply_column_comments(spark, table_name, comments=comments, verbose=True)

## 🥪 Part 2: The "Polars Sandwich" (Spark ➡️ Polars ➡️ Spark)
When the data is small enough for a single node, flip it to Polars for high-performance local processing, then seamlessly send it back to Spark. `VolumeSpiller` handles the I/O automatically via Unity Catalog.

In [0]:
# Step 1: Spark to Polars (Converts and loads into Driver memory)
# Using cleanup=False keeps the temporary files in volume after reading
# Using eager=True reads as Dataframe, =False scans as Lazyframe
pl_df = spill.spark_to_polars(df_clean, cleanup=False, eager=True)

print(f"⚡ Converted to Polars. Shape: {pl_df.shape}")

# Step 2: Do heavy lifting in Polars 
# Let's drop the duplicates we found earlier and add a new column
pl_processed = (
    pl_df
    .unique(subset=["User_ID"])
    .with_columns(
        pl.col("First_Name").str.to_uppercase().alias("First_Name_Upper")
    )
)
print(pl_processed.head())

# Step 3: Polars to Spark
# Automatically fixes datetime precision (ns/us -> ms) and attaches UTC timezone
# Polars defaults to ns/us, can cause read issue with spark
spark_final = spill.polars_to_spark(pl_processed, cleanup=False)

print("🚀 Converted back to Spark!")
spark_final.show()

## 💾 Part 3: Checkpointing & Hybrid Storage Options
Stop recalculating the same data. Save intermediate states to the Unity Catalog Volume (persistent) or the local driver disk (ephemeral/fast).

In [0]:
# 1. Save Polars DataFrame to Volume (Persistent)
spill.save_checkpoint_pl(pl_processed, name="gold_users_vol", storage="volume")

# 2. Save Polars DataFrame to Local Driver /tmp (Ephemeral, wiped on cluster restart)
spill.save_checkpoint_pl(pl_processed, name="gold_users_local", storage="local")

# 3. Save a PySpark DataFrame checkpoint
spill.save_checkpoint_spark(spark_final, name="spark_checkpoint")

print("\n📂 Current Checkpoints:")
print(f"  Volume Storage: {spill.list_checkpoints('volume')}")
print(f"  Local Storage:  {spill.list_checkpoints('local')}")

# Reloading to verify it works
reloaded_pl = spill.load_checkpoint_pl("gold_users_local", storage="local")
print(f"\n✅ Reloaded {reloaded_pl.height} rows from Local checkpoint.")

## 🧹 Part 4: Cleanup
The `teardown()` method behaves differently based on your `IS_DEV` flag.
- **In Dev (`is_dev=True`)**: Leaves the Volume intact so you can inspect the Parquet files in the Catalog Explorer.
- **In Prod (`is_dev=False`)**: Drops the volume completely to avoid clutter and storage costs.

In [0]:
print(f"Cleanup mode: {'Preserving data for inspection' if IS_DEV else 'Dropping Volume completely'}")
spill.teardown()

# Drop the temporary delta table we created for the comments showcase
spark.sql(f"DROP TABLE IF EXISTS {table_name}")
print("Showcase complete! 🎉")